# Utils

> Add description here

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import sys
import pandas as pd
import geopandas as gpd
import numpy as np
import rioxarray as rxr
import xarray as xr
import matplotlib.pyplot as plt
from itertools import product
from pathlib import Path
import shapely
import dask.array as da

import rasterio as rio
from rasterio.enums import Resampling


In [ ]:
#| export

def set_cli_styles():
    "Set styles for logging"
    if sys.platform == 'win32':
        green = red = blue = BOLD = RESET = ""
    else:
        green = "\033[92m"
        red = "\033[91m"
        blue = "\033[94m"
        BOLD = "\x1B[1m"
        RESET = "\x1b[0m"
    return green, red, blue, BOLD, RESET

def format_cli_info(message, green="", red="", blue="", BOLD="", RESET=""):
    return BOLD + green + red + blue + message + RESET

In [ ]:
#| export

def save_raster(x:xr.Dataset, name:Path, crs:str):
    "Save xarray dataset `x` as raster to `name`, using `crs`"
    if not np.all(x == np.zeros_like(x)):
        x.rio.to_raster(name, compress='LZW', crs=crs, tiled=True, windowed=True)

In [ ]:
#| export

def create_qgis_colormap(outname:Path|str, classes:list[str], cm:str='tab20'):
    "Create a qgis colormap .txt -file to `outname` base don `classes`"
    cmap = plt.get_cmap(cm)
    with open(outname, "w") as f:
        for i, classname in enumerate(classes):
            rgb = list((np.array(cmap(i)) * 255).astype(np.uint8))
            f.write(str(i) + " ")
            f.writelines([str(x) + " " for x in rgb])
            f.write(str(classname) + "\n")

In [ ]:
#| export

def create_grid(xmin:float, ymin:float, xmax:float, ymax:float, cellsize:int) -> list[shapely.geometry.Polygon]:
    "Create a regular grid with `cellsize`x`cellsize` squares covering the provided extent"

    cells = [shapely.geometry.box(x, y, x-cellsize, y+cellsize) for x, y in
             product(np.arange(xmin, xmax+cellsize, cellsize), 
                     np.arange(ymin, ymax+cellsize, cellsize))]
    return cells

In [ ]:
#| export

def check_cell(Fx:xr.Dataset, cell:shapely.geometry.Polygon) -> bool:
    "Check whether a location of `cell` is empty in `Fx`"
    try:
        Ax = Fx.rio.clip([cell], from_disk=True)
        return not da.all(Ax == da.zeros_like(Ax)).compute()
    except NoDataInBounds:
        return False
    except ValueError:
        return False

In [ ]:
#| export

def build_overviews(fname:Path|str, factors:list[int]=None):
    "Build overviews for raster located in `fname`, using `factors`. Default is factors from 2 to 512"
    if not factors: 
        factors = [2**(n+1) for n in range(9)]
    dst = rio.open(fname, 'r+')
    dst.build_overviews(factors, Resampling.nearest)
    dst.update_tags(ns='rio_overview', resampling='nearest')
    dst.close()
    return